In [5]:
from datetime import datetime, timedelta
from enum import Enum
from abc import ABC, abstractmethod

class PaymentGateway:
    def process_payment(self, user, amount):
        # Placeholder for payment processing logic
        print(f"Processing payment of {amount} for {user.name}")
        return True

class Discount(ABC):
    @abstractmethod
    def apply_discount(self, amount):
        pass

class PercentageDiscount(Discount):
    def __init__(self, percentage):
        self.percentage = percentage

    def apply_discount(self, amount):
        discount_amount = (self.percentage / 100) * amount
        return amount - discount_amount

class FixedAmountDiscount(Discount):
    def __init__(self, discount_amount):
        self.discount_amount = discount_amount

    def apply_discount(self, amount):
        return max(0, amount - self.discount_amount)

class TripType(Enum):
    FLIGHT = "Flight"
    HOTEL = "Hotel"
    CAR_RENTAL = "Car Rental"
    ACTIVITY = "Activity"

class BookingManager:
    def __init__(self):
        self.booked_trips = []

    def validate_trip_availability(self, trip):
        for booked_trip in self.booked_trips:
            if self._trips_overlap(booked_trip, trip):
                raise ValueError("Trip overlaps with an existing booking.")

        self.booked_trips.append(trip)

    def _trips_overlap(self, trip1, trip2):
        return (
            (trip1.start_date <= trip2.start_date <= trip1.end_date) or
            (trip1.start_date <= trip2.end_date <= trip1.end_date) or
            (trip2.start_date <= trip1.start_date <= trip2.end_date) or
            (trip2.start_date <= trip1.end_date <= trip2.end_date)
        )

class User:
    def __init__(self, user_id, name, email):
        self.user_id = user_id
        self.name = name
        self.email = email
        self.bookings = []
        self.payment_gateway = PaymentGateway()
        self.discounts = []

    def add_discount(self, discount):
        self.discounts.append(discount)

    def book_trip(self, trip, booking_manager):
        try:
            booking_manager.validate_trip_availability(trip)
            # Perform booking logic, integrate with payment gateway, etc.
            booking_amount = trip.cost
            for discount in self.discounts:
                booking_amount = discount.apply_discount(booking_amount)

            booking_status = self.payment_gateway.process_payment(self, booking_amount)
            if booking_status:
                self.bookings.append(trip)
                print(f"Trip booked successfully for {self.name}!")
        except Exception as e:
            print(f"Booking failed for {self.name}: {str(e)}")

class Trip:
    def __init__(self, trip_type, destination, start_date, end_date, cost=0):
        self.trip_type = trip_type
        self.destination = destination
        self.start_date = start_date
        self.end_date = end_date
        self.cost = cost

    def display_details(self):
        print(f"{self.trip_type.value} Details: {self.destination} ({self.start_date} to {self.end_date}), Cost: {self.cost}")

class Flight(Trip):
    def __init__(self, airline, departure_city, arrival_city, departure_time, arrival_time, cost=0):
        super().__init__(TripType.FLIGHT, f"{departure_city} to {arrival_city}", departure_time, arrival_time, cost=cost)
        self.airline = airline
        self.departure_city = departure_city
        self.arrival_city = arrival_city
        self.departure_time = departure_time
        self.arrival_time = arrival_time

    def display_details(self):
        super().display_details()
        print(f"Flight Details: {self.airline}, Departure: {self.departure_city} ({self.departure_time}) to {self.arrival_city} ({self.arrival_time})")

class Hotel(Trip):
    def __init__(self, name, location, check_in_date, check_out_date, cost_per_night):
        super().__init__(TripType.HOTEL, f"{name} in {location}", check_in_date, check_out_date, cost=0)
        self.name = name
        self.location = location
        self.check_in_date = check_in_date
        self.check_out_date = check_out_date
        self.cost_per_night = cost_per_night

    def display_details(self):
        super().display_details()
        print(f"Hotel Details: {self.name} in {self.location}, Check-in: {self.check_in_date}, Check-out: {self.check_out_date}, Cost per Night: {self.cost_per_night}")

class CarRental(Trip):
    def __init__(self, company, pickup_location, return_location, pickup_date, return_date, cost_per_day):
        super().__init__(TripType.CAR_RENTAL, f"{company} Car Rental ({pickup_location} to {return_location})", pickup_date, return_date, cost=0)
        self.company = company
        self.pickup_location = pickup_location
        self.return_location = return_location
        self.pickup_date = pickup_date
        self.return_date = return_date
        self.cost_per_day = cost_per_day

    def display_details(self):
        super().display_details()
        print(f"Car Rental Details: {self.company}, Pickup: {self.pickup_location} ({self.pickup_date}) to {self.return_location} ({self.return_date}), Cost per Day: {self.cost_per_day}")

class Activity(Trip):
    def __init__(self, name, location, date, cost):
        super().__init__(TripType.ACTIVITY, f"{name} in {location}", date, date, cost=0)
        self.name = name
        self.location = location
        self.date = date

    def display_details(self):
        super().display_details()
        print(f"Activity Details: {self.name} in {self.location}, Date: {self.date}, Cost: {self.cost}")

class TravelPackage(Trip):
    def __init__(self, name, start_date, end_date, cost=0):
        super().__init__(TripType.FLIGHT, name, start_date, end_date, cost=cost)
        self.trip_components = []

    def add_trip_component(self, trip_component):
        self.trip_components.append(trip_component)
        self.cost += trip_component.cost

    def display_details(self):
        print(f"\nTravel Package: {self.destination} ({self.start_date} to {self.end_date}), Total Cost: {self.cost}")
        for trip_component in self.trip_components:
            trip_component.display_details()

class Itinerary:
    def __init__(self):
        self.trip_components = []
        self.cost = 0

    def add_trip_component(self, trip_component):
        self.trip_components.append(trip_component)
        self.cost += trip_component.cost

    def display_itinerary(self):
        print("\nComplete Itinerary:")
        for trip_component in self.trip_components:
            trip_component.display_details()


def calculate_total_cost(itinerary):
    return itinerary.cost

def calculate_total_cost(itinerary):
    total_cost = 0
    for trip_component in itinerary.trip_components:
        total_cost += trip_component.cost
    return total_cost

# Example Usage
user1 = User(user_id=1, name="Akshay", email="Akshay@gmail.com")
booking_manager = BookingManager()

percentage_discount = PercentageDiscount(10)
fixed_discount = FixedAmountDiscount(50)

user1.add_discount(percentage_discount)
user1.add_discount(fixed_discount)

flight1 = Flight(
    airline="Air France",
    departure_city="New York",
    arrival_city="Paris",
    departure_time=datetime(2024, 5, 1, 8, 0),
    arrival_time=datetime(2024, 5, 1, 12, 0),
    cost= 30000 
)

hotel1 = Hotel(
    name="Luxury Hotel",
    location="Paris",
    check_in_date=datetime(2024, 5, 1),
    check_out_date=datetime(2024, 5, 10),
    cost_per_night=2000
)
hotel1.cost = (hotel1.check_out_date - hotel1.check_in_date).days * hotel1.cost_per_night  # Calculate cost

car_rental1 = CarRental(
    company="Hertz",
    pickup_location="Paris",
    return_location="Nice",
    pickup_date=datetime(2024, 5, 3),
    return_date=datetime(2024, 5, 7),
    cost_per_day=500
)
car_rental1.cost = (car_rental1.return_date - car_rental1.pickup_date).days * car_rental1.cost_per_day  # Calculate cost

activity1 = Activity(
    name="Guided Tour",
    location="Paris",
    date=datetime(2024, 5, 4),
    cost=300
)

itinerary = Itinerary()
itinerary.add_trip_component(flight1)
itinerary.add_trip_component(hotel1)
itinerary.add_trip_component(car_rental1)
itinerary.add_trip_component(activity1)

user1.book_trip(itinerary, booking_manager)

# Display user bookings and complete itinerary
print(f"\n{user1.name}'s Bookings:")
for booking in user1.bookings:
    if isinstance(booking, Itinerary):
        booking.display_itinerary()
    else:
        booking.display_details()

# Calculate total cost of the trip
total_cost = calculate_total_cost(itinerary)
print(f"\nTotal Cost of the Trip: {total_cost}")


Processing payment of 44950.0 for Akshay
Trip booked successfully for Akshay!

Akshay's Bookings:

Complete Itinerary:
Flight Details: New York to Paris (2024-05-01 08:00:00 to 2024-05-01 12:00:00), Cost: 30000
Flight Details: Air France, Departure: New York (2024-05-01 08:00:00) to Paris (2024-05-01 12:00:00)
Hotel Details: Luxury Hotel in Paris (2024-05-01 00:00:00 to 2024-05-10 00:00:00), Cost: 18000
Hotel Details: Luxury Hotel in Paris, Check-in: 2024-05-01 00:00:00, Check-out: 2024-05-10 00:00:00, Cost per Night: 2000
Car Rental Details: Hertz Car Rental (Paris to Nice) (2024-05-03 00:00:00 to 2024-05-07 00:00:00), Cost: 2000
Car Rental Details: Hertz, Pickup: Paris (2024-05-03 00:00:00) to Nice (2024-05-07 00:00:00), Cost per Day: 500
Activity Details: Guided Tour in Paris (2024-05-04 00:00:00 to 2024-05-04 00:00:00), Cost: 0
Activity Details: Guided Tour in Paris, Date: 2024-05-04 00:00:00, Cost: 0

Total Cost of the Trip: 50000
